In [1]:
from google.colab import drive 
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta
from dateutil import parser
from sklearn.model_selection import train_test_split
from operator import itemgetter

In [3]:
df_c = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/binned_customers.csv', index_col=False)
df_t = pd.read_csv('/content/gdrive/MyDrive/Colab Notebooks/small_transactions1.csv', index_col=False)

In [4]:
del df_t['Unnamed: 0']
del df_c['Unnamed: 0']

In [ ]:
df_t 

,customer_id,article_id,count,binned_age
0,00000dbacae5abe5e23885899a1fa44253a17956c6d1c3...,568601043,1,4.0
1,000058a12d5b43e67d225668fa1f8d618c13dc232df0ca...,794321007,1,1.0
2,0000757967448a6cb83efb3ea7a3fb9d418ac7adf2379d...,448509014,1,1.0
3,0000757967448a6cb83efb3ea7a3fb9d418ac7adf2379d...,719530003,1,1.0
4,00009d946eec3ea54add5ba56d5210ea898def4b46c685...,516859008,1,5.0
...,...,...,...,...
1020507,ffffbbf78b6eaac697a8a5dfbfd2bfa8113ee5b403e474...,804992033,1,1.0
1020508,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,689365050,1,1.0
1020509,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,762846027,1,1.0
1020510,ffffcf35913a0bee60e8741cb2b4e78b8a98ee5ff2e6a1...,794819001,1,1.0


In [5]:
train, test = train_test_split(df_t, test_size=0.1)
print(train.shape)
print(test.shape)

(918460, 4)
(102052, 4)


In [6]:
train = train.reset_index()
test = test.reset_index()

In [ ]:
train.iloc[0]['customer_id']

'dbe124d03b1b8ae22ec7293ece6347546c802b07d46a422d39ce785ea57f20fc'

In [7]:
pip install surprise

     |████████████████████████████████| 11.8 MB 4.6 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1630192 sha256=128c7f352b5d8a2f9ea72a1c11629bf03352fbab50a53ae2e63d2389166e9825
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [8]:
from surprise import SVD
from surprise import accuracy
from surprise.reader import Reader
from surprise.dataset import Dataset
from surprise.model_selection.validation import cross_validate
from surprise.model_selection.search import GridSearchCV
from collections import defaultdict

In [9]:
reader = Reader()
data = Dataset.load_from_df(train[['customer_id', 'article_id', 'count']], reader)

In [ ]:
'''params = {"n_epochs":[30],
          "lr_all":[0.002, 0.005],
          "reg_all":[0.4, 0.6]}'''

In [ ]:
'''clf = GridSearchCV(SVD, params, n_jobs=-1, measures=['rmse'])
clf.fit(data)             
print(clf.best_score)
print(clf.best_params)
print(clf.best_estimator)'''

{'rmse': 0.44877291583311774}
{'rmse': {'n_epochs': 30, 'lr_all': 0.005, 'reg_all': 0.4}}
{'rmse': <surprise.prediction_algorithms.matrix_factorization.SVD object at 0x7ff2ca7d2250>}


In [ ]:
'''svd = clf.best_estimator['rmse']'''

In [10]:
svd = SVD()

In [11]:
data_train = data.build_full_trainset()
resultant_matrix = svd.fit(data_train) 

In [13]:
df_grouped = df_t.groupby(['article_id', 'binned_age']).size().to_frame(name = 'count').reset_index()

In [14]:
df_grouped.sort_values(by=['count'], ascending=False)

,article_id,binned_age,count
89933,915526001,1.0,1272
90842,918292001,1.0,931
17139,706016001,1.0,849
2238,448509014,1.0,817
24461,751471001,1.0,790
...,...,...,...
44534,822126003,3.0,1
44535,822126003,4.0,1
44536,822127004,1.0,1
44537,822127004,3.0,1


In [15]:
temp = []
age_pop = []
for i in range(8):
  temp.append(df_grouped[df_grouped['binned_age'] == i+1].sort_values(by=['count'], ascending=False)['article_id'][:4].tolist())
  age_pop.append(temp[i])

In [16]:
age_pop

[[915526001, 918292001, 706016001, 448509014],
 [918292001, 898694001, 915529003, 909370001],
 [915529003, 751471001, 865929003, 863595006],
 [751471001, 751471043, 915529003, 863595006],
 [896152002, 751471043, 751471001, 896169002],
 [933706001, 751471043, 896152002, 896169002],
 [751471043, 933706001, 896169002, 918522001],
 [865929003, 237347045, 848324001, 875856004]]

In [17]:
unique_a = train['article_id'].unique()
unique_a.shape #28629

(28584,)

In [18]:
unique_c = test['customer_id'].unique()
unique_c.shape

(76684,)

In [19]:
def recommend(age_pop, unique_a, test, train, model):
  recs = []
  for idx in range(test.shape[0]):
    uid = test.iloc[idx]['customer_id']
    if uid in train['customer_id']:
      age_bin = test[test['customer_id']==uid]['binned_age'].to_list()
      rec = age_pop[age_bin[0]-1]
    else:
      pred = []
      for a_id in unique_a:
        pred.append([a_id, model.predict(uid=uid,
                iid=a_id, r_ui=None).est])
      pred.sort(key = lambda x: x[1], reverse=True)
      a = pred[:4]
      rec = list(map(itemgetter(0), a))
    recs.append([uid, rec])
  return recs


In [20]:
test1 = test[:1000]

In [21]:
test1

,index,customer_id,article_id,count,binned_age
0,537088,86a9230d46a615ca0d22955a702c5ea2678584024e279c...,749974001,1,1.0
1,284490,474679b04da72fe115349116e8b6dac415e67d6ee24469...,898596004,1,4.0
2,662154,a5789fc4ffb50b6d169c70c451e882735bf8e28c70bdb6...,892558004,2,1.0
3,553686,8ab2816e4f55649c23f0138e7c6a09f7d3021e3cf8f1e6...,407354004,1,3.0
4,934753,ea73393c3b9b4e08652377a946b6a4af5a870231fe7344...,878604002,1,1.0
...,...,...,...,...,...
995,448969,7093d642e3085501f648c2828b707b1f29f9b2eb82196e...,552716001,1,1.0
996,368694,5c8677a7908c9c3f500831ad3ea70c0e54f20d5da26ae4...,579541001,1,5.0
997,979602,f5a622d0d10ada1d6ccf919657d2eeeee6b9d63a6ca456...,901955001,2,1.0
998,424674,6a59871da918227567804084a7c5ca94e75c123ac8bf60...,837810001,1,1.0


In [22]:
recs = recommend(age_pop, unique_a, test1, train, model=svd)
recs

[['86a9230d46a615ca0d22955a702c5ea2678584024e279c6bb653c09a07af242d', [717464001, 158340001, 850917003, 715255013]], ['474679b04da72fe115349116e8b6dac415e67d6ee2446978074c1619c1ca8d6d', [714032003, 717464001, 715255013, 846407001]], ['a5789fc4ffb50b6d169c70c451e882735bf8e28c70bdb69a0ae641359fdab2a7', [715255013, 918443002, 846407001, 871638002]], ['8ab2816e4f55649c23f0138e7c6a09f7d3021e3cf8f1e6aa026630c03705aa4a', [717464001, 158340001, 846407001, 820866001]], ['ea73393c3b9b4e08652377a946b6a4af5a870231fe7344b1af5f1fe8f325e894', [715255013, 158340001, 846407001, 717464001]], ['a8dc3f6db29c03fe190d82e4ee4acc72525c43fff9eaa92f429d1c471b6e7fb7', [717464001, 846407001, 714032003, 902175001]], ['7b8e32f0f14d4e5b6d984e6656b9086a963a1e01ab6075f466f751a675052a7a', [717464001, 714032003, 846407001, 570002100]], ['8a1b25f181ef3dc6f047511fd93e8c410d5b830a3ef6320274b0ca32ff846c07', [717464001, 158340001, 846407001, 570002100]], ['f55585dc277b3af68a76d0c9b19dcef274d2ce9f31d9ea27e1e05bfe57ce377e', [7

In [ ]:
num_t = df_t.groupby(['customer_id']).size().to_frame(name = 'num_t').reset_index()
num_t

In [56]:
final_error = 0
for idx in range(test1.shape[0]):
  c_id = test1.iloc[idx]['customer_id']
  b = num_t[num_t['customer_id']==c_id]['num_t'].to_list()[0]
  c = df_t[df_t['customer_id']==c_id]
  a = df_t[df_t['customer_id']==c_id]
  true_guess = 0
  for i in range(4):
    for j in range(b):
      if (list(map(itemgetter(0), recs))[i]) == c.iloc[j]['article_id']:
        true_guess = true_guess+1
        break
      else:
        pass

  if b >= 4:
    final_error = final_error + (4-true_guess)/4
  else:
    final_error = final_error + (b-true_guess)/b

print(final_error)



1000.0
